In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,Dense,SimpleRNN

In [2]:
# Loading Data
max_features = 10000
(X_train, y_train),(X_test,y_test) = imdb.load_data(num_words=max_features)
print(f"X_Train: {X_train.shape} X_Test: {X_test.shape}")
print(f"y_Train: {X_train.shape} y_Test: {X_test.shape}")

17464789/17464789 [==============================] - 0s 0us/step
X_Train: (25000,) X_Test: (25000,)
y_Train: (25000,) y_Test: (25000,)


In [3]:
X_train[0]

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 2,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 2,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 2,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 2,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 2,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 2,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5,
 144,
 30,
 5535,
 18,

In [4]:
print(len(X_train[31]), len(X_train[1]))

647 189


In [5]:
maxlen = len(X_train[0])
for arr in np.concatenate((X_train, X_test)):
    lenarr = len(arr)
    if maxlen < lenarr:
        maxlen = lenarr
        msg = arr

print(maxlen)
# Out of curiosity, checking what the message contains
word_index = imdb.get_word_index()
msg = ''
for word_v in arr:
    word_k = next((k for k, v in word_index.items() if v == word_v-3), "?")
    msg = msg+" "+word_k
print(msg)

2494
1641221/1641221 [==============================] - 0s 0us/step
 ? a good ol' boy film is almost required to have ? car chases a storyline that has a vague resemblance to plot and at least one very pretty country gal ? with short shorts and a low top the pretty gal is here dressed in designer ? but the redneck ? stop there jimmy dean is a natural as a ? ? but as a tough guy former sheriff he comes up way short big john is big but he isn't convincing with the bad part of his ? bug eyed jack ? is a hoot as always and bo hopkins has been playing this same part for decades ned beatty also does his part in a small role but there is no story it ? more like an episode of in the heat of the night than a feature film ? with easily predictable sentiment perhaps the most glaring problem with this movie is charlie daniels singing the theme you know the one it was made famous by jimmy dean


In [6]:
X_train = sequence.pad_sequences(X_train, padding="post", maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, padding="post", maxlen=maxlen)
X_train[0][-20:]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int32)

In [ ]:
# TPU Initialization
import tensorflow as tf
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
print('TPU initialized:', tpu)

In [11]:
strategy = tf.distribute.TPUStrategy(resolver)

In [12]:
# Model Building
with strategy.scope():
  model = Sequential()
  model.add(Embedding(max_features,128))
  model.add(SimpleRNN(128, activation='relu'))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer ="adam", loss= "binary_crossentropy",metrics = ["accuracy"])

In [13]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(patience=5, monitor="val_loss",restore_best_weights=True)
es

In [14]:
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2, callbacks=[es])

Epoch 1/10
313/313 [==============================] - 174s 542ms/step - loss: 0.6934 - accuracy: 0.5015 - val_loss: 0.6932 - val_accuracy: 0.4938
Epoch 2/10
313/313 [==============================] - 166s 530ms/step - loss: 0.6933 - accuracy: 0.5009 - val_loss: 0.6932 - val_accuracy: 0.4938
Epoch 3/10
313/313 [==============================] - 166s 529ms/step - loss: 1.9196 - accuracy: 0.4976 - val_loss: 0.6931 - val_accuracy: 0.5062
Epoch 4/10
313/313 [==============================] - 166s 529ms/step - loss: 0.6932 - accuracy: 0.4934 - val_loss: 0.6932 - val_accuracy: 0.4938
Epoch 5/10
313/313 [==============================] - 166s 529ms/step - loss: 0.6932 - accuracy: 0.5006 - val_loss: 0.6931 - val_accuracy: 0.5062
Epoch 6/10
313/313 [==============================] - 166s 530ms/step - loss: 0.6932 - accuracy: 0.4967 - val_loss: 0.6932 - val_accuracy: 0.4938
Epoch 7/10
313/313 [==============================] - 166s 530ms/step - loss: 0.6932 - accuracy: 0.4954 - val_loss: 0.6932 -

In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 128)         1280000   
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 128)               32896     
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1313025 (5.01 MB)
Trainable params: 1313025 (5.01 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
model.save("sentiment_rnn.keras")